In [1]:
import pandas as pd
# import re
from datetime import datetime
import string

In [2]:
# Correção para linhas que começam com múltiplos espaços

def correctSplit(x):
    splitted_str = list(x)
    if splitted_str[0] == '|':
        splitted_str[0] = ''
    return ''.join(splitted_str)

# Função para separar colunas do df importado

def separar_colunas_df(df_raw):
    df = df_raw.copy()
    # lista alfabetica e.g ['a','b','c'...]
    lista_alfabeto = list(string.ascii_lowercase)
    # Pega a primeira coluna do df (que tem todos os dados) e substitui onde possui mais de um espaço por | 
    df.iloc[:,0] = df.iloc[:,0].replace({'( ){2,} ':'|'},regex=True)
    # corrige a substituição para linhas que começam com espaços
    df.iloc[:,0] = df.iloc[:,0].apply(lambda x: correctSplit(x))
    # corta a partir do separador '|' e define cada pedaço para uma coluna ['a','b'..]
    for i in range (len(df.iloc[:,0][0].split('|'))):
        df[lista_alfabeto[i]] = df.iloc[:,0].apply(lambda x: x.split('|')[i])
    # deleta a coluna que originou as demais colunas    
    df.drop(df.columns[0], axis = 1, inplace = True)
    return df

# veic_2019_02.csv_amostra

In [40]:
colnames = ['index','n','x','tipo','data','dia','mes','ano','hora','y','z','vel1','vel2','k','s','cod','placa','r','t','y','u','i']

df_veic = pd.read_csv('amostras/veic_2019_02.csv_amostra.txt')
df_veic.head(5)

,index,n,x,tipo,data,dia,mes,ano,hora,y,...,vel2,k,s,cod,placa,r,t,y.1,u,i
0,1,501,1,2019-02-01 00:00:07,2019-02-01,1,2,2019,00:00:07,0,...,68.0,2,3.8,bdac468f1,LRV5999,0,0,NaN,NaN,NaN
1,2,502,2,2019-02-01 00:00:09,2019-02-01,1,2,2019,00:00:09,0,...,57.0,2,3.8,bdac469a2,LRW4982,0,0,NaN,NaN,NaN
2,3,502,2,2019-02-01 00:00:17,2019-02-01,1,2,2019,00:00:17,0,...,61.0,2,3.6,bdac46ee2,LQM9654,0,0,NaN,NaN,NaN
3,4,501,1,2019-02-01 00:00:20,2019-02-01,1,2,2019,00:00:20,0,...,73.0,2,3.8,bdac470d1,LMR8029,0,0,NaN,NaN,NaN
4,5,501,1,2019-02-01 00:00:29,2019-02-01,1,2,2019,00:00:29,0,...,56.0,3,6.4,bdac476a1,NaN,0,0,NaN,NaN,NaN


In [43]:
df_veic2019 = pd.DataFrame()

df_veic2019['DATAHORA'] = df_veic['tipo']
df_veic2019['PLACA'] = df_veic['placa']
df_veic2019['VELOCIDADE'] = df_veic['vel2']
#CODCET nao consegui identificar nas colunas 
#CGCT nao consegui identificar nas colunas
#LATITUDE pendente de dados
#LONGITUDE pendente de dados

# Salvando em csv
df_veic2019.to_csv('amostras_tratadas/amostra_veic2019_tratada.csv', sep=';', index=False)
df_veic2019.sample(5)

,DATAHORA,PLACA,VELOCIDADE
31,2019-02-01 00:00:30,LPA9711,56.0
13,2019-02-01 00:00:59,LTK2036,70.0
33,2019-02-01 00:00:41,NaN,44.0
56,2019-02-01 00:00:21,KXH4332,81.0
29,2019-02-01 00:00:50,HOI7991,48.0


# movimento_OCR_JAN_2019.txt_amostra.txt

In [6]:
# Nome para coluna inicial
colnames = ['x']

# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_ocr_raw = pd.read_csv('amostras/movimento_OCR_JAN_2019.txt_amostra.txt', sep='|', names = colnames)

df_ocr_split = separar_colunas_df(df_ocr_raw.copy())
df_ocr_split.head()

,a,b,c,d,e
0,KPV68771AUTOMÓVEL,20190101000000971024,0101201900000003900CET8-5455 0410191112025201...,St R Catete,B-22.93380-43.18294
1,LQR82471AUTOMÓVEL,20190101000001181044,0101201900000103400CET8-5410 0410121122025201...,St Tanque,D-22.97019-43.41377
2,ETP18901AUTOMÓVEL,20190101000001281014,0101201900000104000CET8-5436 0410241111025201...,St Cidade Nova,A-22.93149-43.20888
3,KXL99161AUTOMÓVEL,20190101000001381014,0101201900000102500CET8-5439 0410161111025201...,St Colonia,A-22.92479-43.38907
4,LPU22871AUTOMÓVEL,20190101000001901024,0101201900000104400CET8-5401 0410381122025201...,St Pontal,B-23.01267-43.38900


# Tratando OCR

In [38]:
def get_datahora(string):
    date = string[38:52]
    try: 
        date = datetime.strptime(date, "%Y%m%d%H%M%S")
    except:
        date = 'erro'
    return date

def get_velocidade(string):
    vel = string[92:95]
    try:
        vel = int(vel)
    except:
        vel = 'erro'
    return vel

def get_placa(string):
    placa = string[0:7]
    return placa.strip()

df_ocr = pd.DataFrame()
df_ocr['CODCET'] = df_ocr_raw['x'].apply(lambda x: x[108:118])
df_ocr['DATAHORA'] = df_ocr_raw['x'].apply(lambda x: get_datahora(x))
df_ocr['PLACA'] = df_ocr_raw['x'].apply(lambda x: get_placa(x))
df_ocr['VELOCIDADE'] = df_ocr_raw['x'].apply(lambda x: get_velocidade(x))
df_ocr['CGT'] = df_ocr_raw['x'].apply(lambda x: x[108:111])
df_ocr['LATITUDE'] = df_ocr_raw['x'].apply(lambda x: x[216:225])
df_ocr['LONGITUDE'] = df_ocr_raw['x'].apply(lambda x: x[225:234])

# Salvando em csv
df_ocr.to_csv('amostras_tratadas/amostra_movimento_tratada.csv', sep=';', index=False)
df_ocr.sample(5)

,CODCET,DATAHORA,PLACA,VELOCIDADE,CGT,LATITUDE,LONGITUDE
7,0410021111,2019-01-01 00:00:02,,23,041,-22.79752,-43.19290
99,0410311111,2019-01-01 00:00:46,,57,041,-22.88907,-43.56765
77,0410551111,2019-01-01 00:00:34,SUN9684,25,041,-23.00850,-43.43940
70,0410501111,2019-01-01 00:00:28,LSV0784,33,041,-22.98147,-43.58786
42,0410351111,2019-01-01 00:00:17,KXK4498,29,041,-23.01567,-43.41202


# SCP_TRANSCARIOCA_0222016_201903_amostra.txt

In [9]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_transcarioca_raw = pd.read_csv('amostras/SCP_TRANSCARIOCA_0222016_201903_amostra.txt', sep='|', names = colnames)
            
# Separando as colunas
df_transcarioca_split = separar_colunas_df(df_transcarioca_raw.copy())
df_transcarioca_split.head(5)

,a,b,c,d,e,f
0,-------2carro de passeio,010320190010319,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"3-22,87280-43,33629"
1,-------1moto,0103201900104424,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"2-22,87280-43,33629"
2,-------2carro de passeio,0103201900104419,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"3-22,87280-43,33629"
3,-------1moto,0103201900104628,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"3-22,87280-43,33629"
4,-------1moto,0103201900104830,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"2-22,87280-43,33629"


# Tratamento SCP_TRANSCARIOCA

In [42]:
def get_datahora(string):
    date = string[78:92]
    try: 
        date = datetime.strptime(date, "%d%m%Y%H%M%S")
    except:
        date = 'erro'
    return date

def get_placa(string):
    placa = string[:7]
    if placa[0].isalpha():
        pass
    else:
        placa = ''
    return placa

def get_velocidade(string):
    vel = string[92:94]
    try:
        vel = int(vel)
    except:
        vel = 'erro'
    return vel

def get_latitude(string):
    lat = string[216:219]+"."+string[220:225]
    return lat

def get_longitude(string):
    lon = string[225:228]+"."+string[229:]
    return lon

df_transcarioca = pd.DataFrame()
df_transcarioca['CODCET'] = df_transcarioca_raw['x'].apply(lambda x:x[108:118])
df_transcarioca['DATAHORA'] = df_transcarioca_raw['x'].apply(lambda x: get_datahora(x))
df_transcarioca['PLACA'] = df_transcarioca_raw['x'].apply(lambda x: get_placa(x))
df_transcarioca['VELOCIDADE'] = df_transcarioca_raw['x'].apply(lambda x: get_velocidade(x))
df_transcarioca['CGCT'] = df_transcarioca_raw['x'].apply(lambda x:x[108:111])
df_transcarioca['LATITUDE'] = df_transcarioca_raw['x'].apply(lambda x: get_latitude(x))
df_transcarioca['LONGITUDE'] = df_transcarioca_raw['x'].apply(lambda x: get_longitude(x))

# Salvando em csv
df_transcarioca.to_csv('amostras_tratadas/amostra_transcarioca_tratada.csv', sep=';', index=False)
df_transcarioca.head(20)

,CODCET,DATAHORA,PLACA,VELOCIDADE,CGCT,LATITUDE,LONGITUDE
0,0440021120,2019-03-01 00:10:31,,9,044,-22.87280,-43.33629
1,0440021120,2019-03-01 00:10:44,,24,044,-22.87280,-43.33629
2,0440021120,2019-03-01 00:10:44,,19,044,-22.87280,-43.33629
3,0440021120,2019-03-01 00:10:46,,28,044,-22.87280,-43.33629
4,0440021120,2019-03-01 00:10:48,,30,044,-22.87280,-43.33629
5,0440021120,2019-03-01 00:10:50,,30,044,-22.87280,-43.33629
6,0440021120,2019-03-01 00:10:58,,34,044,-22.87280,-43.33629
7,0440021120,2019-03-01 00:11:07,,32,044,-22.87280,-43.33629
8,0440021120,2019-03-01 00:11:08,,33,044,-22.87280,-43.33629
9,0440021120,2019-03-01 00:11:11,,34,044,-22.87280,-43.33629


# SPLICE_007201820190205.txt_amostra.txt

In [15]:
# Nome para coluna inicial
colnames = ['x']

# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_splice_raw = pd.read_csv('amostras/SPLICE_0072018_20190205_.txt_amostra.txt', sep='|', names = colnames)

# Para facilitar o tratamento
df_splice_raw.iloc[:,0] = df_splice_raw.iloc[:,0].replace({'( ){2,} ':'|'},regex=True) #df.iloc[(range linhas),(range colunas)]
df_splice_raw.iloc[:,0] = df_splice_raw.iloc[:,0].apply(lambda x: correctSplit(x))
df_splice_raw.head()

df_splice_raw.head()

,x
0,1MOTO|2019010100000105900RD3389|04503811120072...
1,1MOTO|2019010100000305500RD3381|00450541110072...
2,1MOTO|2019010100000505800RD3384|04504411120072...
3,1MOTO|2019010100002203900RD3414|04506811110072...
4,1MOTO|2019010100002205800RD3385|04504511120072...


In [27]:
df_splice = pd.DataFrame()

def get_datahora(string):
    date = string.split('|')[1][:13]
    try: 
        date = datetime.strptime(date, "%d%m%Y%H%M%S")
    except:
        date = 'erro'
    return date


def get_Lat(string):
    lat_long = string.split('|')[-1]
    lat = '-' + lat_long.split('-')[1]
    return lat

def get_Long(string):
    lat_long = string.split('|')[-1]
    long = '-'+lat_long.split('-')[2]
    return long

df_splice = df_splice_raw.copy()
df_splice['TIPO'] = df_splice_raw.iloc[:,0].apply(lambda x: x.split('|')[0][:1])
df_splice['DATAHORA'] = df_splice_raw.iloc[:,0].apply(lambda x: get_datahora(x))
df_splice['CODCET'] = df_splice_raw.iloc[:,0].apply(lambda x: x.split('|')[2][:10]) 
df_splice['VELOCIDADE'] = df_splice_raw.iloc[:,0].apply(lambda x: x.split('|')[1][15:17])
df_splice['LATITUDE'] = df_splice_raw.iloc[:,0].apply(lambda x: get_Lat(x))
df_splice['LONGITUDE'] = df_splice_raw.iloc[:,0].apply(lambda x: get_Long(x))
df_splice.drop(columns=['x'],inplace=True)
df_splice.to_csv('amostras_tratadas/amostra_splice_tratada.csv', sep=';', index=False)
df_splice.head()
# não foi encontrada nenhuma fatia que possuisse a placa do veículo, tampouco uma exclusiva do CGT
# perguntas: 1- é possível não haver placa?
#           2- o CGT é sempre igual aos primeiros 3 dígitos do CODCET?

,TIPO,DATAHORA,CODCET,VELOCIDADE,LATITUDE,LONGITUDE
0,1,9010-01-20 10:00:00,0450381112,59,-22.98468,-43.21814
1,1,9010-01-20 10:00:00,0045054111,55,-22.95759,-43.19957
2,1,9010-01-20 10:00:00,0450441112,58,-22.98537,-43.20337
3,1,9010-01-20 10:00:02,0450681111,39,-22.98475,-43.20046
4,1,9010-01-20 10:00:02,0450451112,58,-22.98481,-43.20567


# FISCALTECH_0042016_06_2019.txt_amostra.txt

In [30]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_fiscaltech_raw = pd.read_csv('amostras/FISCALTECH_0042016_06_2019.txt_amostra.txt', sep='|', names = colnames)
df_fiscaltech = separar_colunas_df(df_fiscaltech_raw)


# Extraindo partes onde possui informação requisitada
df_fiscaltech['b0'] = df_fiscaltech.b
df_fiscaltech['d'] = df_fiscaltech.c # aqui existe latidude e longitude, além de um número prévio que também poderia sar o 
#código do tipo
df_fiscaltech['b'] = df_fiscaltech.b0.apply(lambda x: x.split(' ')[0])
df_fiscaltech['c'] = df_fiscaltech.b0.apply(lambda x: x.split(' ')[2][:17])

df_fiscaltech.head(4)

,a,b,c,b0,d
0,1,0106201903270901800FSCII2175,04200611130042016,0106201903270901800FSCII2175 0420061113004201...,3-23000409-43334161
1,1,0106201901164305100FSCII2214,04201712110042016,0106201901164305100FSCII2214 0420171211004201...,1
2,1,0106201901120105000FSCII2174,04200711240042016,0106201901120105000FSCII2174 0420071124004201...,4-23001615-43328645
3,1,0106201903304702100FSCII2175,04200611140042016,0106201903304702100FSCII2175 0420061114004201...,4-23000409-43334161


# Tratamento FISCALTECH

In [31]:
def get_datahora(string):
    date = string[:13]
    try: 
        date = datetime.strptime(date, "%d%m%Y%H%M%S")
    except:
        date = 'erro'
    return date


def get_lat(string):
    if len(string.split('-')) == 1: #se estamos no brasil, Lat e Long são <0. Então esses campos estão preenchidos, há '-'
        return ''
    else:
        return '-'+string.split('-')[1][:2]+'.'+string.split('-')[1][2:]

def get_long(string):
    if len(string.split('-')) == 1: #se estamos no brasil, Lat e Long são <0. Então esses campos estão preenchidos, há '-'
        return ''
    else:
        return '-'+string.split('-')[2][:2]+'.'+string.split('-')[2][2:]

def get_placa(string):
    if len(string)==1:
        return ''
    else:
        return string[:7]

def get_tipo(string):
    if len(string)==1:
        return string
    else:
        return string[7:8]


#Separação de colunas 

df_fiscaltech['PLACA'] = df_fiscaltech.a.apply(lambda x: get_placa(x))
df_fiscaltech['TIPO'] = df_fiscaltech.a.apply(lambda x: get_tipo(x))
df_fiscaltech['CODCET'] = df_fiscaltech.c.apply(lambda x: x[:10])
df_fiscaltech['CGT'] = df_fiscaltech.c.apply(lambda x: x[:3])
df_fiscaltech['DATAHORA'] = df_fiscaltech.b.apply(lambda x: get_datahora(x))
df_fiscaltech['VELOCIDADE'] = df_fiscaltech.b.apply(lambda x: x[15:17]) #o certo seria [14:19], mas estamos padronizando
#sem casa decimal
df_fiscaltech['LATITUDE'] = df_fiscaltech.d.apply(lambda x: get_lat(x))
df_fiscaltech['LONGITUDE'] = df_fiscaltech.d.apply(lambda x: get_long(x)) #perceber que deixo um número antes do - de fora

df_fiscaltech.drop(['a','b','c','d','b0'], axis = 1, inplace = True)

# Salvando em csv
df_fiscaltech.to_csv('amostras_tratadas/amostra_fiscaltech_tratada.csv', sep=';', index=False)

# Print
df_fiscaltech.sample(10)

,PLACA,TIPO,CODCET,CGT,DATAHORA,VELOCIDADE,LATITUDE,LONGITUDE
44,,1,0420071222,042,2019-06-01 03:43:01,62,-23.001678,-43.328644
8,,1,0420071113,042,2019-06-01 03:33:00,58,-23.00141,-43.328091
57,,1,0420061223,042,2019-06-01 03:20:04,46,-23.000813,-43.334781
20,,1,0420021123,042,2019-06-01 03:26:02,39,-23.000226,-43.353781
51,,1,0420061112,042,2019-06-01 01:15:00,02,-23.000409,-43.334161
97,,1,0420121113,042,2019-06-01 03:19:00,55,-22.967687,-43.397829
52,,1,0420061114,042,2019-06-01 01:15:01,27,-23.000409,-43.334161
18,,1,0420081112,042,2019-06-01 06:19:02,23,-23.00303,-43.32414
39,,1,0420071223,042,2019-06-01 03:22:00,13,-23.001678,-43.328644
21,,1,0420141122,042,2019-06-01 03:33:02,43,-22.981046,-43.409698


# SCP_RADAR_RIO_SITRAN_0232015_201903_amostra.txt

In [36]:
#Tratamento da SITRAN
# Nome para coluna inicial
colnames = ['x']

# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_sitran_raw = pd.read_csv('amostras/SCP_RADAR_RIO_SITRAN_0232015_201903_amostra.txt', sep='|', names = colnames)

df_sitran_split = separar_colunas_df(df_sitran_raw.copy())
df_sitran_split.head()

,a,b,c,d,e,f
0,-------1moto,0103201910263053,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO,"1-22,90393-43,27340"
1,-------1moto,0103201910263141,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO,"2-22,90393-43,27340"
2,-------1moto,0103201910263449,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO,"3-22,90393-43,27340"
3,LQS56581moto,0103201910490948,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO,"1-22,90393-43,27340"
4,FLT20441moto,0103201910491046,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO,"3-22,90393-43,27340"


In [37]:
def get_datahora(string):
    date = string[78:92]
    try: 
        date = datetime.strptime(date, "%d%m%Y%H%M%S")
    except:
        date = 'erro'
        
    return date

def get_velocidade(string):
    vel = string[92:95]
    try:
        vel = int(vel)
    except:
        vel = 'erro'
    return vel


def get_placa(string):
    placa = string[:7]
    if placa[0].isalpha():
        pass
    else:
        placa = ''
    return placa

df_sitran = pd.DataFrame()
df_sitran['CODCET'] = df_sitran_raw['x'].apply(lambda x: x[108:118])
df_sitran['DATAHORA'] = df_sitran_raw['x'].apply(lambda x: get_datahora(x))
df_sitran['PLACA'] = df_sitran_raw['x'].apply(lambda x: get_placa(x))
df_sitran['VELOCIDADE'] = df_sitran_raw['x'].apply(lambda x: get_velocidade(x))
df_sitran['CGT'] = df_sitran_raw['x'].apply(lambda x: x[108:111])
df_sitran['LATITUDE'] = df_sitran_raw['x'].apply(lambda x: x[216:225])
df_sitran['LONGITUDE'] = df_sitran_raw['x'].apply(lambda x: x[225:234])

# Salvando em csv
df_sitran.to_csv('amostras_tratadas/amostra_sitran_tratada.csv', sep=';', index=False)
df_sitran.sample(5)

,CODCET,DATAHORA,PLACA,VELOCIDADE,CGT,LATITUDE,LONGITUDE
39,0400141110,2019-03-01 10:51:19,,54,040,"-22,90393","-43,27340"
66,0400141110,2019-03-01 10:53:16,,48,040,"-22,90393","-43,27340"
75,0400141110,2019-03-01 10:53:36,LRX5110,39,040,"-22,90393","-43,27340"
89,0400141110,2019-03-01 10:54:05,,51,040,"-22,90393","-43,27340"
4,0400141110,2019-03-01 10:49:10,FLT2044,46,040,"-22,90393","-43,27340"


# SCP_INTVIA_0182018_201901_amostra.txt

In [32]:
#Tratamento de INTVIA
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_intvia_raw = pd.read_csv('amostras/SCP_INTVIA_0182018_201901_amostra.txt', sep='|', names = colnames)

# separando as colunas
df_intvia_split = separar_colunas_df(df_intvia_raw)

df_intvia_split.head(5)

,a,b,c,d,e,f
0,-------1moto,0101201900000215,BRI712,04800911100182018Rua Jardim Botanico,GAVEA,"1-22,96635-43,21941"
1,LSY84841moto,0101201900000417,BRI712,04800911100182018Rua Jardim Botanico,GAVEA,"2-22,96635-43,21941"
2,-------1moto,0101201900000714,BRI712,04800911100182018Rua Jardim Botanico,GAVEA,"1-22,96635-43,21941"
3,-------1moto,0101201900002229,BRI712,04800911100182018Rua Jardim Botanico,GAVEA,"1-22,96635-43,21941"
4,-------1moto,0101201900004443,BRI712,04800911100182018Rua Jardim Botanico,GAVEA,"1-22,96635-43,21941"


In [33]:
def get_placa(string):
    placa = string[:7]
    if placa[0].isalpha():
        pass
    else:
        placa = ''
    return placa

def get_datahora(string):
    date = string[78:92]
    try: 
        date = datetime.strptime(date, "%d%m%Y%H%M%S")
    except:
        date = 'erro'
    return date
def get_velocidade(string):
    vel = string[92:95]
    try:
        vel = int(vel)
    except:
        vel = 'erro'
    return vel


df_intvia = pd.DataFrame()
df_intvia['CODCET'] = df_intvia_raw['x'].apply(lambda x: x[108:118])
df_intvia['DATAHORA'] = df_intvia_raw['x'].apply(lambda x: get_datahora(x))
df_intvia['PLACA'] = df_intvia_raw['x'].apply(lambda x: get_placa(x))
df_intvia['VELOCIDADE'] = df_intvia_raw['x'].apply(lambda x: get_velocidade(x))
df_intvia['CGT'] = df_intvia_raw['x'].apply(lambda x: x[108:111])
df_intvia['LATITUDE'] = df_intvia_raw['x'].apply(lambda x: x[216:225])
df_intvia['LONGITUDE'] = df_intvia_raw['x'].apply(lambda x: x[225:234])


df_intvia.to_csv('amostras_tratadas/amostra_intvia_tratada.csv', sep=';', index=False)
df_intvia.sample(5)

,CODCET,DATAHORA,PLACA,VELOCIDADE,CGT,LATITUDE,LONGITUDE
49,0480091110,2019-01-01 00:13:01,,46,048,"-22,96635","-43,21941"
43,0480091110,2019-01-01 00:10:32,QMR3894,25,048,"-22,96635","-43,21941"
6,0480091110,2019-01-01 00:00:52,,39,048,"-22,96635","-43,21941"
91,0480091110,2019-01-01 00:24:34,,24,048,"-22,96635","-43,21941"
52,0480091110,2019-01-01 00:14:15,,43,048,"-22,96635","-43,21941"
